In [1]:
import json

with open('artist_dict.json', 'r') as file:
    artist_dict = json.load(file)

In [29]:
from pyvis.network import Network

net = Network(directed=True, height='750px', width='1200px')
for artist_uri in artist_dict:
    line_counter = 0
    genres = ''
    for genre in artist_dict[artist_uri]['genres']:
        if (len(f'{genres}{genre}, ') // 54) == line_counter:
            genres = f'{genres}{genre}, '
        else:
            line_counter += 1
            genres = f'{genres}<br>{genre}, '
    genres = genres[:-2]
    net.add_node(
        artist_uri,
        label=artist_dict[artist_uri]['name'],
        title=('<b>Name</b>: ' + artist_dict[artist_uri]['name'] +
               '<br><b>URI</b>: ' + artist_uri +
               '<br><b>Spotify</b>: ' + artist_dict[artist_uri]['url'] +
               '<br><b>Popularity</b>: ' + 
               str(artist_dict[artist_uri]['popularity']) +
               '<br><b>Followers</b>: ' + 
               str(artist_dict[artist_uri]['followers']) +
               '<br><b>Genres</b>: ' + genres))
            
for artist_uri in artist_dict:
    if 'related_artists' in artist_dict[artist_uri]:
        for related_artist_uri in artist_dict[artist_uri]['related_artists']:
            net.add_edge(artist_uri, related_artist_uri)

min_frequency = 0
while min_frequency < 6:
    edge_dict = {}
    for edge in net.edges:
        if edge['from'] not in edge_dict:
            edge_dict[edge['from']] = set()
        edge_dict[edge['from']].add(edge['to'])
        if edge['to'] not in edge_dict:
            edge_dict[edge['to']] = set()
        edge_dict[edge['to']].add(edge['from'])
    for node in net.nodes:
        if node['id'] not in edge_dict:
            edge_dict[node['id']] = set()
    frequency_counter = {node: len(edge_dict[node]) for node in edge_dict}
    min_frequency = min(frequency_counter.values())
    nodes_to_remove = {node for node in frequency_counter if 
                       frequency_counter[node] == min_frequency}
    net.nodes = [node for node in net.nodes if node['id'] not in 
                 nodes_to_remove]
    net.edges = [edge for edge in net.edges if len(set(
        [edge['to'], edge['from']]).intersection(nodes_to_remove)) == 0]
    
    
for node in net.nodes:
    artist_uri = node['id']
    if 'related_artists' in artist_dict[artist_uri]:
        node['title'] += ('<br><b>Related Artists</b>: <br>' + '<br>'.join(
            [artist_dict[related_artist_uri]['name'] for related_artist_uri in 
             artist_dict[artist_uri]['related_artists']]))
        
    if len(artist_dict[artist_uri]['images']) > 0:
        node['image'] = artist_dict[artist_uri]['images'][0]
        node['shape'] = 'circularImage'
    
reverse_relation_dict = {}
for edge in net.edges:
    if edge['to'] not in reverse_relation_dict:
        reverse_relation_dict[edge['to']] = set()
    reverse_relation_dict[edge['to']].add(edge['from'])
    
for node in net.nodes:
    if node['id'] in reverse_relation_dict:
        node['title'] += ('<br><b>Suggested By</b>: <br>' + '<br>'.join(
            [artist_dict[related_artist_uri]['name'] for 
             related_artist_uri in 
             sorted(reverse_relation_dict[node['id']])]))

net.barnes_hut(gravity=-40000, central_gravity=0.15, overlap=1)
net.show('related_artist_network.html')